In [1]:
import pandas as pd
import os
import glob

In [22]:
folder_path = '../data/RLWM_examples'
# Get a list of all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

for csv_file in csv_files:
    #print(f"Processing {csv_file}")
    df = pd.read_csv(csv_file)

    # Filter rows where 'trial_mode' is 'learning'
    df = df.loc[df['trial_mode'] == 'learning']
    # Remove "blockId" column
    df = df.drop(columns=['blockId'])
    # Create block number - block incremented by 1 for each new setId / setSize
    df['block'] = ((df['setId'] != df['setId'].shift()) | (df['setSize'] != df['setSize'].shift())).cumsum() - 1
    # Rename stimId based on block and setsize
    df['stimseq'] = df.groupby('block')['stimId'].transform(lambda x: pd.factorize(x)[0])
    # Calculate iteration numbers
    df['iterseq'] = df.groupby(['block', 'stimId']).cumcount() + 1

    # Select specific columns from df
    selected_columns = df[['block', 'setSize', 'stimseq', 'stimId', 'setId', 'iterseq']]
    # Rename the columns
    new_df = selected_columns.rename(columns={
        'setSize': 'ns',
        'stimId': 'imageseq',
        'setId': 'folderseq'
    }).astype(int)


# Descriptions about new_df structure
#??? 1. subno - subject ID
# 2. block - block number (there is something wrong with the first block, too many zeros…)
# 3. ns - set size
#??? 4. Time – tbd
# 5. stimseq - stimulus id (transform real stimID to match setsize limit)
# 6. imageseq - actual image number
# 7. folderseq - image category number
# 8. iterseq - iteration number per stimulus

In [21]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None) 
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

new_df.to_csv('../data/preprocessed/new.csv', index=False)
new_df

,block,ns,stimseq,imageseq,folderseq,iterseq
0,0,2,0,3,12,1
1,0,2,1,5,12,1
2,0,2,0,3,12,2
3,0,2,0,3,12,3
4,0,2,1,5,12,2
...,...,...,...,...,...,...
380,10,4,2,2,9,9
381,10,4,1,4,9,10
382,10,4,3,3,9,10
383,10,4,2,2,9,10
